## RHOFOLD+ experimentation

In [28]:
import pandas as pd
from torch.utils.data import Dataset, DataLoader
import numpy as np
import torch
import os
from tqdm import tqdm

PATH_DATA = "../data/"
INPUT_DIR = "input_fasta_test"
RHOFOLD_PATH = "../models/RhoFold/"

In [3]:
test_data = pd.read_csv(PATH_DATA + "test_sequences.csv")

In [14]:
test_data.head()
print("Test data has length:", len(test_data))

Test data has length: 12


In [13]:
class RNADataset(Dataset):
    def __init__(self,data):
        self.data=data
        self.tokens={nt:i for i,nt in enumerate('ACGU')}
        
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        sequence = self.data.loc[idx,'sequence']
        tokenized_sequence = np.array([self.tokens[nt] for nt in sequence])
        tokenized_sequence = torch.tensor(tokenized_sequence)
        return tokenized_sequence
    
dataset = RNADataset(test_data)
print("Tokenized sequence:", dataset[0])

Tokenized sequence: tensor([2, 2, 2, 2, 2, 1, 1, 0, 1, 0, 2, 1, 0, 2, 0, 0, 2, 1, 2, 3, 3, 1, 0, 1,
        2, 3, 1, 2, 1, 0, 2, 1, 1, 1, 1, 3, 2, 3, 1, 0, 2, 1, 1, 0, 3, 3, 2, 1,
        0, 1, 3, 1, 1, 2, 2, 1, 3, 2, 1, 2, 0, 0, 3, 3, 1, 3, 2, 1, 3])


In [25]:
#generate directory for fasta files 
os.makedirs(INPUT_DIR, exist_ok=True)

for target_id in tqdm(test_data["target_id"].values, desc="Generating input fasta files"):
    sequence = test_data[test_data["target_id"] == target_id]["sequence"].to_string(index = False, header = False)
    with open(f"{INPUT_DIR}/{target_id}.fasta", "w") as text_file:
        text_file.write(f">{target_id}\n")
        text_file.write(sequence)

Generating input fasta files: 100%|██████████| 12/12 [00:00<00:00, 527.60it/s]


In [29]:
for target_id in test_data["target_id"].values:
    os.makedirs(f"./out/{target_id}", exist_ok = True)
    msa_a3m_file = f"{PATH_DATA}+/MSA/{target_id}.MSA.fasta"
    input_fasta_file = f"{INPUT_DIR}/{target_id}.fasta"
    if len(open(input_fasta_file, 'r').read()) > 400:
        print(f"Skipping {target_id} because it's too long")
        continue
    device = "cpu" if len(open(input_fasta_file, 'r').read()) > 400 else "cuda:0"
    run_folding = f"python inference.py --relax_steps 0 --input_fas {input_fasta_file} --input_a3m {msa_a3m_file} --output_dir ./out/{target_id}/ --device {device} --ckpt {RHOFOLD_PATH}/pretrained/RhoFold_pretrained.pt"
    os.system(run_folding)

python: can't open file '/home/ignasi/Desktop/kagle/rhofold_experimentation/inference.py': [Errno 2] No such file or directory
python: can't open file '/home/ignasi/Desktop/kagle/rhofold_experimentation/inference.py': [Errno 2] No such file or directory
python: can't open file '/home/ignasi/Desktop/kagle/rhofold_experimentation/inference.py': [Errno 2] No such file or directory
python: can't open file '/home/ignasi/Desktop/kagle/rhofold_experimentation/inference.py': [Errno 2] No such file or directory
python: can't open file '/home/ignasi/Desktop/kagle/rhofold_experimentation/inference.py': [Errno 2] No such file or directory
python: can't open file '/home/ignasi/Desktop/kagle/rhofold_experimentation/inference.py': [Errno 2] No such file or directory
python: can't open file '/home/ignasi/Desktop/kagle/rhofold_experimentation/inference.py': [Errno 2] No such file or directory
python: can't open file '/home/ignasi/Desktop/kagle/rhofold_experimentation/inference.py': [Errno 2] No such fi